In [1]:
import ML_ADMM_jpeg_corner_smoothing as mlcsc

import tensorflow as tf
import pickle as pkl
import jpeg_related_functions as jrf
import numpy as np

rho = 1.
noi = 32
alpha_init = 1.5
blkSmthCoef = 1.0
pklfile = 'ML_LRA_checkpoint_epoch_128.ckpt.pkl'
num_of_epochs = 1
#steps_per_epoch = 0

rho = 1.
noi = 32
alpha_init = 1.5
blkSmthCoef = 1.0
n_components = 4
cmplxdtype = tf.complex128 # This should really be elsewhere.
batch_size = 1
#steps_per_epoch = 4
step_size = 0.01
#num_of_epochs = 2

#   ******** BASE NAMES AND DIRECTORIES  ********
modelname = 'ML_ADMM_'
databasename = 'BSDS500/'
#databasename = 'simpleTest/'
experimentname = 'experiment1/'

#   ******** DEPENDENT NAMES AND DIRECTORIES
experimentpath = 'data/experiment/' + databasename + experimentname
checkpointfilename = modelname + 'checkpoint_epoch_{epoch:02d}.ckpt'
timesname = 'times/' + modelname + 'rho' + str(rho) + '_iter' + str(noi) + '_times.pkl'
modelfilename = modelname + 'initial_model.ckpt'

#   ******** DATA AND EXPERIMENT PARAMETERS ********
fid = open(experimentpath + 'problem_param.pckl','rb')
problem_param = pkl.load(fid)
fid.close()
data_param = problem_param['data_param']
targetSz = data_param['target_size']
qY = data_param['qY']
qUV = data_param['qUV']
strides = problem_param['stride']
fltrSz = problem_param['fltrSz']
real_dtype = data_param['dtype']
#noi = problem_param['noi']
noL = problem_param['noL']
noc = problem_param['noc']
datapath = problem_param['datapath']
trainfile = problem_param['trainfile']
valfile = problem_param['valfile']
padding = data_param['padding']
fid = open(experimentpath + pklfile,'rb')
mu_init = pkl.load(fid)
D = pkl.load(fid)
b_init = pkl.load(fid)
fid.close()

#   ******** CROPPING AND PADDING ********
cropAndMerge = mlcsc.CropPadObject(targetSz,strides,[np.asarray(ks) for ks in fltrSz],real_dtype,blkSz = (8,8))
paddingTuple = cropAndMerge.paddingTuple
fftSz = cropAndMerge.get_fft_size(targetSz,strides)
paddingDiff = ((padding[0][0] - paddingTuple[0][0],padding[0][1] - paddingTuple[0][1]),(padding[1][0] - paddingTuple[1][0],padding[1][1] -  paddingTuple[1][1]))
assert(paddingDiff[0][0] >= 0)
assert(paddingDiff[0][1] >= 0)
assert(paddingDiff[1][0] >= 0)
assert(paddingDiff[1][1] >= 0)
    
#   ******** SETUP LOADING TFRECORD ********
startr = paddingDiff[0][0]
startc = paddingDiff[1][0]
endr = targetSz[0] + padding[0][0] + padding[0][1] - paddingDiff[0][1]
endc = targetSz[1] + padding[1][0] + padding[1][1] - paddingDiff[1][1]
example_structure = {'highpass': tf.io.FixedLenFeature([], tf.string), 'lowpass': tf.io.FixedLenFeature([], tf.string), 'compressed': tf.io.FixedLenFeature([], tf.string),'raw': tf.io.FixedLenFeature([], tf.string)}

def restore_double(x):
    return tf.io.parse_tensor(x,real_dtype)

class RGB2Y(tf.keras.layers.Layer):
    def __init__(self,*args,**kwargs):
        super().__init__(*args,**kwargs)
        self.rgb2yuv = jrf.RGB2YUV(dtype = self.dtype)
    def call(self,inputs):
        s_YUV = self.rgb2yuv(inputs)
        return s_YUV[slice(None),slice(None),slice(0,1)]

rgb2y = RGB2Y(dtype=real_dtype)

def _parse_image_function(example_proto):
    x = tf.io.parse_single_example(example_proto, example_structure)
    highpass = restore_double(x['highpass'])
    lowpass = restore_double(x['lowpass'])
    #return ((highpass[slice(startr,endr),slice(startc,endc),slice(None)],lowpass[slice(startr,endr),slice(startc,endc),slice(None)],restore_double(x['compressed'])),rgb2y(restore_double(x['raw'])))
    return ((highpass[slice(startr,endr),slice(startc,endc),slice(None)],lowpass[slice(startr,endr),slice(startc,endc),slice(None)],restore_double(x['compressed'])),restore_double(x['raw']))

raw_dataset = tf.data.TFRecordDataset([datapath + valfile])
dataset = raw_dataset.map(_parse_image_function)
dataset_batch = dataset.batch(batch_size)



print(D[0].shape)
print(D[1].shape)
print(problem_param['D'][0].shape)
print(problem_param['D'][1].shape)

#   ******** BUILD MODEL ********
CSC = mlcsc.MultiLayerCSC_JPEGY(rho,alpha_init,mu_init,b_init,blkSmthCoef,qY,cropAndMerge,fftSz,strides,[tf.expand_dims(D_temp,axis=0) for D_temp in D],n_components,noi,noL,cmplxdtype)


(7, 7, 1, 48)
(5, 5, 48, 64)
(1, 7, 7, 1, 48)
(1, 5, 5, 48, 64)


In [2]:
for (x,y) in dataset_batch:
    x_input = x
    x_output = y
    break



In [3]:
s = CSC.preprocess(x_input)
y,u,By,negC,itstats = CSC.init_vars(s)


In [4]:
x,Ax = CSC.xstep(y,u,By,negC)

In [17]:
augLangBefore = CSC.evaluateLagrangian(s,x,y,u,Ax,By,negC)

representation error:  tf.Tensor(5.159849654548241, shape=(), dtype=float64)
coefficient_penalty:  tf.Tensor(0.010930505507348743, shape=(), dtype=float64)
constraint penalty:  tf.Tensor(10.99363157132818, shape=(), dtype=float64)
u correction:  tf.Tensor(31.3474920182214, shape=(), dtype=float64)


In [18]:
uhalf = CSC.relax(u,Ax,By,negC)

In [19]:
augLangAfter = CSC.evaluateLagrangian(s,x,y,uhalf,Ax,By,negC)
print('Augmented Lagrangian Increase: ',augLangAfter - augLangBefore)

representation error:  tf.Tensor(5.159849654548241, shape=(), dtype=float64)
coefficient_penalty:  tf.Tensor(0.010930505507348743, shape=(), dtype=float64)
constraint penalty:  tf.Tensor(7.5514846077502735, shape=(), dtype=float64)
u correction:  tf.Tensor(18.92563404148522, shape=(), dtype=float64)
Augmented Lagrangian Increase:  tf.Tensor(8.979711013158273, shape=(), dtype=float64)


In [20]:
AugLangBefore = CSC.evaluateLagrangian(s,x,y,uhalf,Ax,By,negC)

representation error:  tf.Tensor(5.159849654548241, shape=(), dtype=float64)
coefficient_penalty:  tf.Tensor(0.010930505507348743, shape=(), dtype=float64)
constraint penalty:  tf.Tensor(7.5514846077502735, shape=(), dtype=float64)
u correction:  tf.Tensor(18.92563404148522, shape=(), dtype=float64)


In [21]:
y,By = CSC.ystep(x,uhalf,Ax,negC)

In [22]:
augLangAfter = CSC.evaluateLagrangian(s,x,y,uhalf,Ax,By,negC)
print('Augmented Lagrangian Decrease: ',augLangBefore - augLangAfter)

representation error:  tf.Tensor(5.076108652130519, shape=(), dtype=float64)
coefficient_penalty:  tf.Tensor(0.01173836921997972, shape=(), dtype=float64)
constraint penalty:  tf.Tensor(7.53088801117616, shape=(), dtype=float64)
u correction:  tf.Tensor(18.92563404148522, shape=(), dtype=float64)
Augmented Lagrangian Decrease:  tf.Tensor(-8.876181277879066, shape=(), dtype=float64)


In [23]:
AugLangBefore = CSC.evaluateLagrangian(s,x,y,uhalf,Ax,By,negC)

representation error:  tf.Tensor(5.076108652130519, shape=(), dtype=float64)
coefficient_penalty:  tf.Tensor(0.01173836921997972, shape=(), dtype=float64)
constraint penalty:  tf.Tensor(7.53088801117616, shape=(), dtype=float64)
u correction:  tf.Tensor(18.92563404148522, shape=(), dtype=float64)


In [24]:
u = CSC.ustep(uhalf,Ax,By,negC)

In [25]:
augLangAfter = CSC.evaluateLagrangian(s,x,y,u,Ax,By,negC)
print('Augmented Lagrangian Increase: ',augLangAfter - augLangBefore)

representation error:  tf.Tensor(5.076108652130519, shape=(), dtype=float64)
coefficient_penalty:  tf.Tensor(0.01173836921997972, shape=(), dtype=float64)
constraint penalty:  tf.Tensor(14.014266696174698, shape=(), dtype=float64)
u correction:  tf.Tensor(7.53088801117616, shape=(), dtype=float64)
Augmented Lagrangian Increase:  tf.Tensor(26.754305993186666, shape=(), dtype=float64)


In [26]:
AugLangBefore = CSC.evaluateLagrangian(s,x,y,u,Ax,By,negC)

representation error:  tf.Tensor(5.076108652130519, shape=(), dtype=float64)
coefficient_penalty:  tf.Tensor(0.01173836921997972, shape=(), dtype=float64)
constraint penalty:  tf.Tensor(14.014266696174698, shape=(), dtype=float64)
u correction:  tf.Tensor(7.53088801117616, shape=(), dtype=float64)


In [27]:
x,Ax = CSC.xstep(y,u,By,negC)

In [28]:
augLangAfter = CSC.evaluateLagrangian(s,x,y,u,Ax,By,negC)
print('Augmented Lagrangian Decrease: ',augLangBefore - augLangAfter)

representation error:  tf.Tensor(0.5594243747965242, shape=(), dtype=float64)
coefficient_penalty:  tf.Tensor(0.01173836921997972, shape=(), dtype=float64)
constraint penalty:  tf.Tensor(4.731309043576802, shape=(), dtype=float64)
u correction:  tf.Tensor(7.53088801117616, shape=(), dtype=float64)
Augmented Lagrangian Decrease:  tf.Tensor(-12.954664063254775, shape=(), dtype=float64)
